In [78]:
import transformers
import pandas as pd
from datasets import load_dataset, load_dataset_builder, Dataset
import torch
import importlib
import numpy as np

In [10]:
import sys
sys.path.append("../../lib")
from juan import *
import utils

In [11]:
data = pd.read_csv("../../data/task_1.csv")
show(data)

(3817, 2)


,sentence,task_1
0,La solución es desarrollar el pensamiento crít...,False
1,Hay que enseñar que la magia no existe.,False
2,Que todos los avances de la humanidad siempre ...,False
3,Enseñar en las escuelas la historia de las rel...,False
4,Desde las religiones de la edad de piedra hast...,False


In [12]:
!pwd

/lustre06/project/6001735/detests_2022/notebooks/juan


In [13]:
model_name = "dccuchile/bert-base-spanish-wwm-uncased"
cache_dir = "/home/jarobyte/.cache"

In [14]:
!ls /home/jarobyte/.cache

fontconfig  huggingface  jedi  matplotlib  pip


In [15]:
%%time
tokenizer = transformers.AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

CPU times: user 50.9 ms, sys: 16.8 ms, total: 67.7 ms
Wall time: 6min


In [16]:
%%time
model = transformers.AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels = 2)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

CPU times: user 747 ms, sys: 687 ms, total: 1.43 s
Wall time: 1min 21s


In [17]:
# dataset = load_dataset(
#     'csv', 
#     data_files={'train': '../../data/task_1.csv'}, 
# #     encoding = "ISO-8859-1"
# )
# dataset

In [18]:
# dataset = Dataset.from_pandas(data)
# dataset

In [19]:
dataset = Dataset.from_dict(dict(sentence = data.sentence, task_1 = data.task_1))
dataset

Dataset({
    features: ['sentence', 'task_1'],
    num_rows: 3817
})

In [20]:
def tokenize_data(example):
    return tokenizer(example['sentence'], padding='max_length')

In [21]:
encoded_dataset = dataset.rename_column("task_1", "label").map(tokenize_data, batched=True)
encoded_dataset#["train"][0]

  0%|          | 0/4 [00:00<?, ?ba/s]

Dataset({
    features: ['sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3817
})

In [65]:
encoded_dataset.set_format(
    type = "torch",
    columns = ['input_ids', 'token_type_ids', 'attention_mask', 'label']
)
encoded_dataset["label"]

tensor([False, False, False,  ..., False, False, False])

In [22]:
dataloader = torch.utils.data.DataLoader(
    encoded_dataset, 
    batch_size = 32
)
# next(iter(dataloader))

In [23]:
next(iter(dataloader))

{'label': tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,  True,
         False, False, False, False, False, False, False, False, False, False,
         False, False]),
 'input_ids': tensor([[    4,  1032,  4667,  ...,     1,     1,     1],
         [    4,  1311,  1041,  ...,     1,     1,     1],
         [    4,  1041,  1399,  ...,     1,     1,     1],
         ...,
         [    4,  9191,  1076,  ...,     1,     1,     1],
         [    4,  1054, 26118,  ...,     1,     1,     1],
         [    4,  1399,  1792,  ...,     1,     1,     1]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1

In [24]:
training_args = transformers.TrainingArguments(
    "test_trainer", 
    num_train_epochs = 3
)

In [25]:
train_dataset = dataset.shuffle(seed=10)

In [26]:
trainer = transformers.Trainer(
    model = model, 
    args = training_args, 
    train_dataset = encoded_dataset#["train"], 
#     eval_dataset=eval_dataset
)

In [45]:
# trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3817
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1434


Step,Training Loss


KeyboardInterrupt: 

In [67]:
pipe = transformers.TextClassificationPipeline(
    model = model, 
    tokenizer = tokenizer,
    return_all_scores = True,
    device = 0
)

In [69]:
labels = pipe(data.sentence.tolist())
labels[:5]

[[{'label': 'LABEL_0', 'score': 0.9999254941940308},
  {'label': 'LABEL_1', 'score': 7.455104787368327e-05}],
 [{'label': 'LABEL_0', 'score': 0.999908447265625},
  {'label': 'LABEL_1', 'score': 9.16014687390998e-05}],
 [{'label': 'LABEL_0', 'score': 0.9999279975891113},
  {'label': 'LABEL_1', 'score': 7.201722473837435e-05}],
 [{'label': 'LABEL_0', 'score': 0.9999270439147949},
  {'label': 'LABEL_1', 'score': 7.299384014913812e-05}],
 [{'label': 'LABEL_0', 'score': 0.999911904335022},
  {'label': 'LABEL_1', 'score': 8.81098021636717e-05}]]

In [84]:
predictions = np.array([l[1]["score"] for l in labels]) > 0.5
predictions[:5]

array([False, False, False, False, False])

In [85]:
(predictions == data.task_1).mean()

0.9986900707361802

In [104]:
class HuggingFaceClassifier:
    def tokenize_data(self, example):
        return tokenizer(example['features'], padding = 'max_length')
    
    def entrenar(self, X, y):
        ds = Dataset.from_dict(
            dict(
                features = X,
                label = y
            )
        )
        ds = ds.map(self.tokenize_data, batched = True)
        ds.set_format(
            type = "torch",
            columns = [
                'input_ids', 
                'token_type_ids', 
                'attention_mask', 
                'label'
            ]
        )
#         dataloader = torch.utils.data.DataLoader(
#             ds, 
#             batch_size = 32
#         )
        training_args = transformers.TrainingArguments(
            "test_trainer", 
            num_train_epochs = 3
        )
        trainer = transformers.Trainer(
            model = model, 
            args = training_args, 
            train_dataset = ds.shuffle()#["train"], 
        #     eval_dataset=eval_dataset
        )
        trainer.train()
        
    def predecir(self, X):
        pipe = transformers.TextClassificationPipeline(
            model = model, 
            tokenizer = tokenizer,
            return_all_scores = True,
            device = 0
        )
        labels = pipe(X.tolist())
        predictions = np.array([l[1]["score"] for l in labels])
        return predictions > 0.5



clf = HuggingFaceClassifier()
# clf.entrenar(data.sentence.values, data.task_1)
# clf.predecir(data.sentence.tolist())

In [105]:
# preds = clf.predecir(data.sentence.tolist())
# preds[:5]

In [106]:
importlib.reload(utils)
results = utils.validate(data.sentence.values, data.task_1.to_frame(), clf, verbose = True)
results

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

fold 0
training model...


  0%|          | 0/4 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: features. If features are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/jarobyte/envs/detests/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3053
  Num Epochs = 3
  Instantaneous batch size per device = 8
  To

...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: features. If features are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/jarobyte/envs/detests/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3054
  Num Epochs = 3
  Instantaneous batch size per device = 8
  To

Step,Training Loss
500,0.191300
1000,0.186600


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




generating predictions on the train set...
generating predictions on the test set...


,test_accuracy,test_f1,test_precision,test_recall,train_accuracy,train_f1,train_precision,train_recall
column,,,,,,,,
task_1,0.975112,0.946733,0.931379,0.963225,0.974523,0.945297,0.929421,0.962117


In [107]:
results.to_csv("../results/task_1/data/beto.csv")